# Retrieval Augmented Generation
In this notebook, we'll learn how to do [Retrieval Augmented Generation](https://www.promptingguide.ai/techniques/rag) with ChromaDB and OpenAI to answer questions about Bunu Denedin Mi Service.

In [1]:
OPEN_AI_API_KEY = "6975730fef866c5e84a0b3848df84e6ac70426651270edaf9bd1ca7c084c2e87"

## Loading Data

In [2]:
import os
from langchain.document_loaders import Docx2txtLoader

document_folder_name = "docs"
documents = os.listdir(document_folder_name)

In [3]:
data = []

for doc_name in documents:
    
    if doc_name.endswith(".docx"):
        
        doc_path = os.path.join(document_folder_name, doc_name)
        loader = Docx2txtLoader(doc_path)
        doc_content = loader.load()
        data.append(doc_content[0])

for document in data:
    document.page_content = document.page_content.replace('\n', '')

In [4]:
data[0]

Document(page_content="Başlık: Hata 0x80070057 2013 veya sonraki sürümlerde Çevrimdışı Outlook Defteri indirilemedi Açıklama: Hata 0x80070057 2013 veya sonraki sürümlerde Çevrimdışı Outlook Defteri indirilemediÇözüm: Yapılandırılmış bir Ertelenmiş Teslim kuralı olup olmadığını ve yapılandırılmışsa kural için yapılandırılmış ölçütleri belirlemek için aşağıdaki adımları kullanın.Outlook 2010 veya sonraki sürümleriŞeritte\xa0Kurallar'a ve sonra Uyarıları\xa0Yönet'e & tıklayın.E-posta Kuralları sekmesinde listelenen\xa0her kuralı\xa0seçin ve kural açıklaması altındaki\xa0bilgileri incelenin.Gönderme sırasında teslimi\xa0geciktiren\xa0kurallar,\xa0kural açıklamasında teslimi # dakika erteler.Kural için belirtilen ölçütleri incelemek üzere herhangi bir kurala çift tıklayın.Gerekirse, teslimi birkaç dakika ertele eylemsini temizin.Bu ayar aşağıdaki şekilde etkin olarak gösterilmektedir. Keywords: E-posta iletileri, Ertelenmiş Teslim Kuralı Nedeniyle Giden Ktusunda KalmasıDoküman oluşturma tar

## Storing embeddings in ChromaDB
Next, let's store those documents as embeddings in ChromaDB

In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [6]:
embeddings = OpenAIEmbeddings(openai_api_key=OPEN_AI_API_KEY)

In [7]:
store = Chroma.from_documents(
    data, 
    embeddings, 
    ids = [f"{item.metadata['source']}-{index}" for index, item in enumerate(data)],
    collection_name="rag_embeddings", 
    persist_directory='db',
)
store.persist()

## Asking questions
Now let's use OpenAI, augmented by ChromaDB, to ask some questions.

In [1]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import pprint

In [9]:
template = """You are a bot that answers questions about Wimbledon 2023, using only the context provided.
If you don't know the answer, simply state that you don't know.

{context}

Question: {question}"""

PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

In [10]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=OPEN_AI_API_KEY)

In [ ]:
ChatOpenAI()

In [11]:
qa_with_source = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=store.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT, },
    return_source_documents=True,
)

In [12]:
response = qa_with_source("Outlook’da imza nasıl oluşturulur?")
print("Sorun:", response['query'] + "\n")
print("Çözüm:", response['result'] + "\n\n\nSorununuzun çözümü aşağıdaki belgede yer almaktadır.\n") 
print("Source:", response['source_documents'][0].metadata['source'])

Sorun: Outlook’da imza nasıl oluşturulur?

Çözüm: Outlook’da imza nasıl oluşturulur?

Çözüm: 1. Adım: Windows 10 masaüstünüzde Outlook'u açın ve Dosya>Seçenekler'e gidin. Seçenekler penceresinde Posta sekmesine gidin ve 'İmzalar' düğmesini tıklayın. 2. Adım: Yeni bir pencere açılacaktır. 'Yeni' düğmesini tıklayın ve imzanıza bir ad verin. Kısa ve kesin tutun. Outlook ile yapılandırılmış birden fazla e-posta hesabınız varsa, imzasını oluşturduğunuz hesabı seçmek için 'E-posta hesabı' açılır menüsünü kullanın. Bu imzalar hesap bazında oluşturulur. Ad oluşturulduktan sonra, altındaki 'İmzaları Düzenle' alanına imzayı girin. 3. Adım: Outlook'ta imza nasıl eklenir? Outlook, varsayılan olarak bir iletiye ayarladığınız imzalardan otomatik olarak bir imza ekler. Ancak, istediğiniz zaman farklı bir tane ile değiştirebilirsiniz. Bir mesajda kullanılan imzayı değiştirmek için, mesaj oluştur penceresindeki Mesaj sekmesindeki 'İmza' açılır menüsünü tıklayın ve kullanmak istediğiniz imzalardan birin

In [13]:
response = qa_with_source("outlook eklentilerimi nasıl yönetebilirim?")
print("Sorun:", response['query'] + "\n")
print("Çözüm:", response['result'] + "\n\n\nSorununuzun çözümü aşağıdaki belgede yer almaktadır.\n") 
print("Source:", response['source_documents'][0].metadata['source'])

Sorun: outlook eklentilerimi nasıl yönetebilirim?

Çözüm: Outlook eklentilerinizi yönetmek için aşağıdaki adımları izleyebilirsiniz:

1. Outlook'u açın.
2. Dosya menüsüne gidin.
3. Outlook Seçenekleri'ni seçin.
4. Eklentiler sekmesine gidin.
5. Eklentilerinizi yönetmek için Git düğmesine tıklayın.
6. Açılan pencerede eklentilerinizi etkinleştirebilir veya devre dışı bırakabilirsiniz.

Bu adımları takip ederek Outlook eklentilerinizi yönetebilirsiniz.


Sorununuzun çözümü aşağıdaki belgede yer almaktadır.

Source: KB_docs\OUTLOOKEKLENTILERININENABLE-DISABLEEDILMESIHAKKINDA1681200993136.docx


In [14]:
response = qa_with_source("Outlook hesabım sürekli office 365 e yönleniyor bunu nasıl durdururum?")
print("Sorun:", response['query'] + "\n")
print("Çözüm:", response['result'] + "\n\n\nSorununuzun çözümü aşağıdaki belgede yer almaktadır.\n") 
print("Source:", response['source_documents'][0].metadata['source'])

Sorun: Outlook hesabım sürekli office 365 e yönleniyor bunu nasıl durdururum?

Çözüm: Outlook hesabınızın sürekli olarak Office 365'e yönlendirilmesini durdurmak için aşağıdaki adımları izleyebilirsiniz:

1. Bilgisayarınızda "Regedit" (Kayıt Defteri Düzenleyicisi) programını açın.
2. Sol taraftaki menüden "HKEY_CURRENT_USER" seçeneğini genişletin.
3. "Software" klasörünü genişletin.
4. "Microsoft" klasörünü genişletin.
5. "Office" klasörünü genişletin.
6. "16.0" klasörünü genişletin.
7. "Common" klasörünü genişletin.
8. "Identity" klasörünü seçin.
9. Sağ tarafta "SignedOutADUser" adında bir kayıt bulunmalıdır. Bu kaydı silin.
10. Windows profil ayarlarına girin.
11. "İş ya da okul erişimi" menüsünden Office 365 bireysel hesabınızı kaldırın.

Bu adımları takip ederek Outlook hesabınızın Office 365'e yönlendirilmesini durdurabilirsiniz.


Sorununuzun çözümü aşağıdaki belgede yer almaktadır.

Source: KB_docs\outlookbaglantİhatasioffice365.docx
